<img src="./images/Banner_NB.png">

# Linear Regression - Teaching Notebook




<a name='top'></a>
In the previous videos we started to learn about linear regression. In this notebook we will learn how to implement it in Python using sklearn. The notebook is divided into 3 parts:

- **Part 1:** to be practiced after video that describes the basic use of linear regression
- **Part 2:** (<a href=#bookmark_p2>quick link</a>) to be practiced after you learned about evaluation methods for linear regression, in particular R^2
- **Part 3:** (<a href=#bookmark_p3>quick link</a>) to be practiced after you learned about non linear regression and more complex regression models



## Part 1 - Simple Linear Regression Implementation

In this part we will focus on the following five main topics:
+ Using the sklearn module for linear regression (`linear_model`)
+ Get familiar with the advertising data set that we use
+ Basic data set visualization for hints for regression results
+ Learn a linear regression model (using `fit` and apply `predict` for forecasting).
+ Investigation of the linear regression model (looking into `coef_` and `intercept`)


### Using sklearn for linear regression

In this course we'll use the linear regression implementation that is part of scikit-learn. There are also other packages (that are not part of this course, such as [statsmodels](http://statsmodels.sourceforge.net/) and [SciPy](http://www.scipy.org/) ).

As usual we will star by importing the relevant modules. Note the `linear_model` import.

In [ ]:
# imports 
import numpy as np
import pandas as pd
from sklearn import linear_model


#and visualization setup
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6)

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

 


### Advertising dataset
To illustrate linear regression, we'll use the 'Advertising' dataset from
[here](http://www-bcf.usc.edu/~gareth/ISL/data.html)


For 200 different ‘markets’ (think different cities), this dataset consists of the number of sales of a particular product as well as the advertising budget for different media: Digital, radio, and newspaper. 

We’ll use linear regression to study the effect of advertising on sales. 
Here, sales is the dependent variable and the budgets are the independent variables. This might help inform or evaluate an advertising strategy for this product.  

In [ ]:
advert = pd.read_csv('./data/Advertising.csv',index_col=0) #load data
advert.head(10)

### Basic data visulaization for hints on regression

First, we will start with initial description of the data (do you know what command?) and then will do some visualization

In [ ]:
advert.describe()

Now let's visualize the data. As you recall from the video lecture, we will start with a scatter plot, with different indication for each advertising channel. 

In [ ]:
plt.scatter(x=advert['Digital'],y=advert['Sales'],c='r',marker='s',label='Digital')
plt.scatter(x=advert['Radio'],y=advert['Sales'],c='b',marker='o',label='Radio')
plt.scatter(x=advert['Newspaper'],y=advert['Sales'],c='k',marker='*',label='Newspaper')

plt.legend(numpoints=1,loc=4)
plt.xlabel('Ad budget (Thousands of dollars)')
plt.ylabel('Sales (units of product)')
plt.show()

What do you see in the data and plot?

#### Observations 
1. From the plot, it is clear that there is a relationship between the advertising budgets and sales. Basically, the more money spent, the larger the number of sales. 
+  The most money was spent on Digital advertising. The amount for Radio and Newspaper is about the same in all markets, whereas the standard deviation for TV advertising is larger. 



#### Questions
1. How can we quantify the relationship between advertising and sales? Can we predict the effect of each ad media on sales? Is the relationship linear? 
+  Which of the different ad media (Digital, Radio, Newspaper) are the most effective at generating sales? 
+  Are there interactions between the different ad media?

First, let's just look at the **effect of Digital advertising on sales**. We use the linear regression model
$$
Sales = \beta_0 + \beta_1 * Digital.
$$


### Build our first linear regression model

Let's build our first linear regression model. For that we will use the `LinearRegression()` class and train the model using it's `fit` function. As mentioned before, we will start by building a model that predicts the sales only based on Digital advertising. Therefore, in the call to  `fit`, we pass as $X$ only the column of $Digital$ (expressed as `advert.iloc[:,0:1]` ). 

In [ ]:
# build our fist linear regression model
m=linear_model.LinearRegression().fit(advert.iloc[:,0:1],advert.iloc[:,3:])


That's it, now in `m` we have our linear regression model. Easy, isn't it?

### Apply the model (using `predict`)
Let's use the model, and see what would the prediction of number of sales based on the Digital advertising budget. We will use the same scatter plot as before to show the actual sales based on Digital advertisng, but also add a line that represents the sales prediction according to the linear regression model. You can see below, how we use the `predict` function of the model, to express the sales prediction.

In [ ]:
plt.scatter(x=advert['Digital'],y=advert['Sales'],c='k',marker='*',label='Digital')
plt.plot(advert['Digital'],m.predict(advert.iloc[:,0:1]),'k',color='blue',linewidth=3)

plt.xlabel('Digital budget (Thousands of dollars)')
plt.ylabel('Sales (Thousand units of product)')
plt.show()

### Investigate the model

But what are the values of  ${\beta}_0$ and ${\beta}_1$



In [ ]:
# let's look inside the model
print("b1:",m.coef_)
print("b0:",m.intercept_)

#### Interpretation and discussion

The intercept of the line is $\hat{\beta}_0 = 7.032$. This means that without any Digital advertising, the model predicts that 7,032 units of product will be sold. 

The slope of the line is $\hat{\beta}_1 = 0.0475$. This means that the model predicts that for every additional $1k spent on TV advertising, an additional 47.5 units of product are sold. 

Are these good results? This is the time to go back to the course site, and watch the next vidoes. Come back to Part 2 of this notebook once you learned the evaluation method videos, and you are referred back to this notebook.

(link to top is <a href="#top">here</a>)

<a name='bookmark_p2'></a>
## Part 2 - Linear Regression - Evaluation Method and multivariate regression

In this part we will focus on the following three main topics:
+ Evaluation metrics for linear regression, including: $SSR$ and $R^2$
+ Linear regression model predicting sales as a function of Radio advertisement or Newspaper advertisement (separate models one for each attribute)
+ Multi-variate linear regression - combing Digital, Newspaper and Radio into a single model

### Evaluation metrics

One way to measure the quality of the fit is to look at the sum of the squared error,
$$
SSE = \sum_{i=1}^n (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i)^2. 
$$

We don't have exactly a method for this in python, we can either calculate it ourselves, as below (or do some post processing to an existing sklearn method called `mean_squared_error` - which we'll show you soon)

In [ ]:
def sse(Y, Y_HAT):  
    sse = sum([(y - y_hat)**2 for y,y_hat in zip(Y, Y_HAT)])
    return sse

SSE = sse(advert.Sales.tolist(),m.predict(advert.iloc[:,0:1]).flatten())

print(SSE)


Alternatively, we can use the method of [`mean_squared_error`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error). However, since it calculates the mean, we need to multiply the output we get with the number of items that we predicted. See below

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(advert.Sales.tolist(),m.predict(advert.iloc[:,0:1]).flatten())*len(advert.Sales.tolist())

We got exactly the same result, which is great! 

Let's analyze the result now. Is this a small or big error?
...good question. This number, SSR, is difficult to interpret by itself. In next slide will see another mehtod.. any idea? 



Exactly... this is the **$R^2$ value** we saw in the lecture, this is a much more interpretable number. Luckily we have such a metric in sklearn, called [`r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score). Later in this section, we will see another way to get the $R^2$ score when building a model (stay tuned for the multivariate regression section..). Anyway, let's see the `r2_score` method


In [ ]:
from sklearn.metrics import r2_score

r2_score(advert.Sales.tolist(),m.predict(advert.iloc[:,0:1]).flatten())


As you can see, in our model, the value is $R^2 = 0.612$, which isn't bad. The model explains $61\%$ of the variability in sales. 



*Note*: for linear regression, the $R^2$ value is the same as correlation, but the $R^2$ value more easily generalizes to more complicated regression models than correlation, so the $R^2$ value is typically considered instead of correlation.

![image](http://imgs.xkcd.com/comics/linear_regression.png)



###  Regression for Radio and Newspaper

Recall that we not only know the ad budget for Digital, but also Radio and Newspaper. 

Next, let's repeat the linear regression analysis for the other types of advertisements. First for Radio and sales

In [ ]:
lr_radio = linear_model.LinearRegression() # create a linear regression object

# scikit-learn doesn't work as well with pandas, so we have to extract values 
x = advert['Radio'].values.reshape(advert['Radio'].shape[0],1)
y = advert['Sales'].values.reshape(advert['Sales'].shape[0],1)

lr_radio.fit(X=x, y=y)

plt.scatter(x, y,  color='black')
plt.plot(x, lr_radio.predict(x), color='blue', linewidth=3)

plt.xlabel('Radio budget (Thousands of dollars)')
plt.ylabel('Sales (Thousand units of product)')
plt.show()

print("Slope:",lr_radio.coef_)
print("Intercept:",lr_radio.intercept_)
print("R2:",lr_radio.score(x,y))

and now for Newspaper and sales

In [ ]:
lr_newspaper = linear_model.LinearRegression() # create a linear regression object

# scikit-learn doesn't work as well with pandas, so we have to extract values 
x = advert['Newspaper'].values.reshape(advert['Newspaper'].shape[0],1)
y = advert['Sales'].values.reshape(advert['Sales'].shape[0],1)

lr_newspaper.fit(X=x, y=y)

plt.scatter(x, y,  color='black')
plt.plot(x, lr_newspaper.predict(x), color='blue', linewidth=3)

plt.xlabel('Newspaper budget (Thousands of dollars)')
plt.ylabel('Sales (Thousand units of product)')
plt.show()

print("Slope:",lr_newspaper.coef_)
print("Intercept:",lr_newspaper.intercept_)
print("R2:",lr_newspaper.score(x,y))

Let's display all three models on a single plot with different colors:

In [ ]:
plt.scatter(x=advert['Digital'],y=advert['Sales'],c='r',marker='s',label='Digital')
plt.scatter(x=advert['Radio'],y=advert['Sales'],c='b',marker='o',label='Radio')
plt.scatter(x=advert['Newspaper'],y=advert['Sales'],c='k',marker='*',label='Newspaper')
plt.legend(numpoints=1,loc=4)

plt.plot(advert['Digital'],m.predict(advert['Digital'].values.reshape(advert['Digital'].shape[0],1)),c='r',linewidth=3)
plt.plot(advert['Radio'],lr_radio.predict(advert['Radio'].values.reshape(advert['Radio'].shape[0],1)),c='b',linewidth=3)
plt.plot(advert['Newspaper'],lr_newspaper.predict(advert['Newspaper'].values.reshape(advert['Newspaper'].shape[0],1)),c='k',linewidth=3)

plt.xlabel('Ad budget (Thousands of dollars)')
plt.ylabel('Sales (units of product)')
plt.show()


What do you think? How do you interpert the results?

#### Interpretation

*So what is the most effective advertising media?*

The slope for radio is the largest, so you might argue that this is the most effective advertising media. For every additional \$1k spent on Radio advertising, an additional 202 units of product are sold. (Compare to 54.7 for newspaper and 47.5 for TV.)

On the other hand, the $R^2$ value for radio is just $33\%$. So the model isn't explaining as much of the data as the model for Digital advertising ($R^2 = 61\%$), but is explaining more than the model for newspaper advertising ($R^2 = 5\%$). 



The main problem with the approach here is that for each advertising media we look at, we're ignoring the ads in the other media. For example, in the model for Digital advertising, 
$$
Sales = \beta_0 + \beta_1 * Digital,
$$
we're ignoring both Radio and Newspaper advertising. 

We need to take all three into account at once. Maybe we can construct a model that looks like 
$$
Sales = \beta_0 + \beta_1 * Digital + \beta_2*Radio + \beta_3*Newspaper. 
$$
This is the idea behind Multiple Linear Regression. 

## Multiple Linear Regression

**Model:**
$$
Sales = \beta_0 + \beta_1 * Digital + \beta_2*Radio + \beta_3*Newspaper. 
$$


We'll start by building and training the model, this time the features ($X$) as you can see has 3 columns (attributes)

In [ ]:
lr = linear_model.LinearRegression() # create a linear regression object

x = advert[['Newspaper',"Radio","Digital"]]
y = advert['Sales']
lr.fit(X=x, y=y);





When we analyze the model, instead of just getting $\beta_0$ and $\beta_1$ as before, we should get now $\beta_1$, $\beta_2$ and $\beta_3$ (for each of the attributes)  - you can see them in the array we get from calling the `coef_` function, in addition to $\beta_0$ that you still get from the `intercept_` command.

In [ ]:
print("Slope:",lr.coef_)
print("Intercept:",lr.intercept_)


In order to evaluate the mode it will be useful to get its $R^2$ value. One option is to use the `r2_score` method, another one is to use the model's `score` [function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score). The `score` function gets the $X$ and $y$ values used to train the regressor, and returns the $R^2$ value. See both functions before, and luckily they both return the same value..  

In [ ]:
print("R2:",lr.score(x,y))
print("R2:",r2_score(y,lr.predict(x.values)))

So.. What is your interpertation of this model with multiple attributes? Is it better than single attrbiute ones? worse? Which attribute is more significant than others?

#### Interpretation

Spending an additional \$1,000  on radio advertising results in an increase in sales by 189 units. Radio is the most effective at method of advertising. 
 

How about the other individual contributors?

Let's look on the correlation between the attributes - maybe we have a confounding effect amonth them. 
Plotted below is also a scatter plot matrix, which is a good way of visualizing the correlations. 

In [ ]:
print(advert.corr())
pd.plotting.scatter_matrix(advert, figsize=(10, 10), diagonal='kde')
plt.show()

The correlation between Newspaper and Radio is 0.35, which implies that in markets where the company advertised using Radio, they also advertised using newspaper. Thus, the influence of Radio on Sales can be incorrectly attributed to Newspaper advertisements! 

This leads us to the following linear regression model, where we forget about Newspaper advertisements:
$$
\text{Sales} = \beta_0 + \beta_1 * \text{Digital_budget} + \beta_2*\text{Radio_budget} 
$$


In [ ]:
lr = linear_model.LinearRegression() # create a linear regression object

# scikit-learn doesn't work as well with pandas, so we have to extract values 
x = advert[["Radio","Digital"]]
y = advert['Sales']

lr.fit(X=x, y=y)


print("Slope:",lr.coef_)
print("Intercept:",lr.intercept_)
print("R2:",lr.score(x,y))

This model performs pretty well. It accounts for $R^2 = 90\%$ of the variance in the data. 

Can we further improve the results? This is the time to go back to the course site, and watch the next vidoes. Come back to Part 3 of this notebook once you learned the non linear regression method videos, and you are referred back to this notebook.

(Link to the <a href="#top">top</a> of the notebook )

<a name='bookmark_p3'> </a>
## Part 3 - Non-Linear Regression 

In this part we will focus on the following two main topics:
+ Nonlinear relationships and feature engineering
+ Overfitting

## Nonlinear relationships

We can consider the interaction between TV and Radio advertising in the model, by taking 


$$
\text{Sales} = \beta_0 + \beta_1 * \text{Digital_budget} + \beta_2*\text{Radio_budget} + \beta_3*\text{Newspaper_budget} + \beta_4 \text{Digital_budget} *\text{Radio_budget}. 
$$


The rational behind the last term is that perhaps spending $x$ on television advertising and $y$ on radio advertising leads to more sales than simply $x+y$. In marketing this is known as the *synergy effect* and in statistics it is known as the *interaction effect*.

**Note**: even though the relationship between the independent and dependent variables is nonlinear, the model is still linear. 

In [ ]:
lr = linear_model.LinearRegression() # create a linear regression object

advert["DigitalxRadio"]=advert["Radio"]*advert["Digital"]


# scikit-learn doesn't work as well with pandas, so we have to extract values 
x = advert[["Radio","Digital","DigitalxRadio"]].values.reshape(advert[["Radio","Digital","DigitalxRadio"]].shape[0],3)
y = advert['Sales'].values.reshape(advert['Sales'].shape[0],1)

lr.fit(X=x, y=y)


print("Slope:",lr.coef_)
print("Intercept:",lr.intercept_)
print("R2:",lr.score(x,y))

This model is really excellent. $R^2 = 97\%$ of the variability in the data is accounted for by the model. 

## A word of caution on overfitting (more on this later)

It is tempting to include a lot of terms in the regression, but this is problematic. A useful model will  *generalize* beyond the data given to it. You may want to come back to this illustration after the next video on overfitting

![image](http://imgs.xkcd.com/comics/curve_fitting_2x.png)
